## Import Lib

In [1]:
!nvidia-smi

Mon Apr 15 10:07:13 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     On  | 00000000:C1:00.0 Off |                    0 |
|  0%   28C    P8              29W / 300W |      4MiB / 46068MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os
import torch
from datasets import load_metric, load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

## Dataset

In [11]:
dataset_name = "mlabonne/guanaco-llama2-1k"
dataset = load_dataset(dataset_name, split="train")
print(type(dataset))

<class 'datasets.arrow_dataset.Dataset'>


## Quantization Config

In [3]:
# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Your GPU supports bfloat16: accelerate training with bf16=True


## Load Model

In [4]:
device_map = {"": 0}
model_name = "NousResearch/Llama-2-7b-chat-hf"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1
print(model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


## Load Token

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

## LoRA Config

In [6]:
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

## Train Config

In [15]:
output_dir = "./results"
num_train_epochs = 1

#set bf16 to True with an A100
fp16 = False
bf16 = True

per_device_train_batch_size = 4

per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

max_grad_norm = 0.3

#AdamW
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

optim = "paged_adamw_32bit"

lr_scheduler_type = "constant"

warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

save_steps = 25

logging_steps = 25

training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
#     max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

## SFT Config

In [16]:
max_seq_length = None
packing = False

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


## Train Model(with Lora)

In [17]:
trainer.train()

/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,1.347900
50,1.617300
75,1.208500
100,1.437700
125,1.177000
150,1.361400
175,1.171900
200,1.459400
225,1.154600
250,1.530300


/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/hice1/jli943/.conda/envs

TrainOutput(global_step=250, training_loss=1.3466120834350586, metrics={'train_runtime': 457.9452, 'train_samples_per_second': 2.184, 'train_steps_per_second': 0.546, 'total_flos': 8755214190673920.0, 'train_loss': 1.3466120834350586, 'epoch': 1.0})

In [18]:
new_model = "llama-2-7b-miniguanaco"
trainer.model.save_pretrained(new_model)

## (Inference) Load base_model + Adapter(LoRA)

In [19]:
# Reload model in FP16 and merge it with LoRA weights
model_lora = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

model_lora = PeftModel.from_pretrained(model_lora, new_model)
model_lora = model_lora.merge_and_unload()
print(id(model_lora))

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

46917617896560


In [22]:
base_model= AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
print(id(base_model))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

46917814564512


In [20]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "你是一位宋词学者，帮助我学习一些宋词相关的知识。"
pipe = pipeline(task="text-generation", model=model_lora, tokenizer=tokenizer, max_length=500)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


<s>[INST] 你是一位宋词学者，帮助我学习一些宋词相关的知识。 [/INST] 宋词是中国古代文学中的一种词语，具有很高的文学价值和历史价值。学习宋词可以帮助我们更好地理解中国古代文学的核心思想和文化传承。

宋词的概念包括：

1. 词语的意义：宋词的意义是指宋词的核心意义，它是宋词的基础和核心。
2. 词语的用法：宋词的用法是指宋词的用法和表达方式，它是宋词的表达形式。
3. 词语的推广：宋词的推广是指宋词的推广和应用，它是宋词的扩展和发展。

学习宋词的知识可以帮助我们更好地理解中国古代文学的核心思想和文化传承，同时也可以帮助我们更好地理解中国古代文学的语言和文学风格。

宋词的知识包括：

1. 宋词的概念和用法：学习宋词的概念和用法，可以帮助我们更好地理解宋词的意义


In [23]:
prompt = "你是一位宋词学者，帮助我学习一些宋词相关的知识。"
pipe = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=500)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] 你是一位宋词学者，帮助我学习一些宋词相关的知识。 [/INST]  Ah, a fellow seeker of knowledge! 😊 As a Song Dynasty scholar, I'd be delighted to share some insights and tips on learning about Song Dynasty-related knowledge.

1. Start with the basics: Begin by familiarizing yourself with the major historical events, cultural practices, and literary works of the Song Dynasty (960-1279 AD). This will provide a solid foundation for your studies.
2. Read the classics: The Song Dynasty was a period of great literary achievement, and many of the works from this era are still studied and admired today. Some of the most famous works include "The Classic of Poetry," "The Book of Odes," and "The Water Margin."
3. Explore the philosophical and political context: The Song Dynasty was marked by a complex web of philosophical and political movements, including Confucianism, Taoism, and Buddhism. Understanding these movements and their impact on society will help you appreciate the cultural context of the time.
4. Stud

In [24]:
# print(model_lora)

# print("--------")
# print(base_model)
# for param_tensor in model_lora.state_dict():
#     print(param_tensor, "\t", model_lora.state_dict()[param_tensor].size())
for param_tensor in model_lora.state_dict():
  if param_tensor == "model.layers.0.self_attn.q_proj.weight":
    # Access first 5 elements (modify as needed)
    first_five_values = model_lora.state_dict()[param_tensor][:5]
    print(param_tensor, ":\n", first_five_values)
    break

for param_tensor in base_model.state_dict():
  if param_tensor == "model.layers.0.self_attn.q_proj.weight":
    # Access first 5 elements (modify as needed)
    first_five_values = base_model.state_dict()[param_tensor][:5]
    print(param_tensor, ":\n", first_five_values)
    break  

model.layers.0.self_attn.q_proj.weight :
 tensor([[-0.0056, -0.0145, -0.0024,  ...,  0.0043,  0.0014, -0.0036],
        [ 0.0145, -0.0042,  0.0028,  ..., -0.0091, -0.0112,  0.0071],
        [-0.0140,  0.0120,  0.0005,  ...,  0.0059,  0.0185, -0.0028],
        [-0.0066,  0.0082,  0.0006,  ...,  0.0043,  0.0093, -0.0092],
        [-0.0089,  0.0151,  0.0013,  ...,  0.0029,  0.0179, -0.0004]],
       device='cuda:0', dtype=torch.float16)
model.layers.0.self_attn.q_proj.weight :
 tensor([[-0.0060, -0.0146, -0.0021,  ...,  0.0042,  0.0018, -0.0035],
        [ 0.0142, -0.0043,  0.0032,  ..., -0.0092, -0.0108,  0.0073],
        [-0.0137,  0.0121,  0.0002,  ...,  0.0061,  0.0181, -0.0030],
        [-0.0064,  0.0083,  0.0003,  ...,  0.0044,  0.0090, -0.0093],
        [-0.0086,  0.0153,  0.0010,  ...,  0.0031,  0.0176, -0.0004]],
       device='cuda:0', dtype=torch.float16)


## Boolq

In [84]:
boolq_dataset = load_dataset("super_glue", "boolq")

boolq_dataset_training=boolq_dataset["train"]
boolq_dataset_validation=boolq_dataset["validation"]
def concatenate_question_passage(example):
#     answer="True" if example["label"]==1 else "False"
    return {'combine': f"<QUESTION>{example['question']} </QUESTION> <PASSAGE> {example['passage']} </PASSAGE>{example['label']}"}

boolq_dataset_training = boolq_dataset_training.map(concatenate_question_passage)
print(boolq_dataset_training[0]["combine"])
# for doc in validation_data:
#     passage = doc["passage"]
#     question = doc["question"]
#     gold_label = doc["label"]
# boolq_dataset_training=list(boolq_dataset_training)
# print(boolq_dataset_training[0])

<QUESTION>do iran and afghanistan speak the same language </QUESTION> <PASSAGE> Persian language -- Persian (/ˈpɜːrʒən, -ʃən/), also known by its endonym Farsi (فارسی fārsi (fɒːɾˈsiː) ( listen)), is one of the Western Iranian languages within the Indo-Iranian branch of the Indo-European language family. It is primarily spoken in Iran, Afghanistan (officially known as Dari since 1958), and Tajikistan (officially known as Tajiki since the Soviet era), and some other regions which historically were Persianate societies and considered part of Greater Iran. It is written in the Persian alphabet, a modified variant of the Arabic script, which itself evolved from the Aramaic alphabet. </PASSAGE>1


In [85]:
training_arguments = TrainingArguments(
    output_dir="./boolq_results",
    num_train_epochs=4,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    #A100 True
    bf16=True,
    max_grad_norm=0.3,
 #   max_steps=max_steps,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)


trainer_boolq = SFTTrainer(
    model=model,
    train_dataset=boolq_dataset_training,
    peft_config=peft_config,
    dataset_text_field="combine",
#     dataset_text_field=["question", "passage"],
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [86]:
trainer_boolq.train()

/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.7508, 'learning_rate': 0.0002, 'epoch': 0.01}


/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.4188, 'learning_rate': 0.0002, 'epoch': 0.02}


/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


...

/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.7109, 'learning_rate': 0.0002, 'epoch': 4.0}


/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'train_runtime': 6260.8206, 'train_samples_per_second': 6.023, 'train_steps_per_second': 1.506, 'train_loss': 1.0528914769119664, 'epoch': 4.0}


TrainOutput(global_step=9428, training_loss=1.0528914769119664, metrics={'train_runtime': 6260.8206, 'train_samples_per_second': 6.023, 'train_steps_per_second': 1.506, 'train_loss': 1.0528914769119664, 'epoch': 4.0})

In [87]:
new_model = "llama-2-7b-boolq"
trainer_boolq.model.save_pretrained(new_model)

In [88]:
# Reload model in FP16 and merge it with LoRA weights
new_model = "llama-2-7b-boolq"
model_lora_boolq = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

model_lora_boolq = PeftModel.from_pretrained(model_lora_boolq, new_model)
model_lora_boolq = model_lora_boolq.merge_and_unload()
print(id(model_lora_boolq))

# Reload tokenizer to save it
# tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

46922335917680


In [89]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = f"<QUESTION>is windows movie maker part of windows essentials</QUESTION> <PASSAGE> Windows Movie Maker -- Windows Movie Maker (formerly known as Windows Live Movie Maker in Windows 7) is a discontinued video editing software by Microsoft. It is a part of Windows Essentials software suite and offers the ability to create and edit videos as well as to publish them on OneDrive, Facebook, Vimeo, YouTube, and Flickr.</PASSAGE>"

pipe = pipeline(task="text-generation", model=model_lora_boolq, tokenizer=tokenizer, max_length=200)
# result = pipe(prompt)
result = pipe(prompt, max_length=1, num_return_sequences=1, do_sample=True)
generated_answer = result[0]['generated_text'][-1]
print(generated_answer)
print(result[0]['generated_text'])

/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


1
<QUESTION>is windows movie maker part of windows essentials</QUESTION> <PASSAGE> Windows Movie Maker -- Windows Movie Maker (formerly known as Windows Live Movie Maker in Windows 7) is a discontinued video editing software by Microsoft. It is a part of Windows Essentials software suite and offers the ability to create and edit videos as well as to publish them on OneDrive, Facebook, Vimeo, YouTube, and Flickr.</PASSAGE>1


In [90]:
boolq_dataset_validation=boolq_dataset["validation"]
def concatenate_question_passage(example):
#     answer="True" if example["label"]==1 else "False"
    return {'combine': f"<QUESTION>{example['question']} </QUESTION> <PASSAGE> {example['passage']} </PASSAGE>"}

boolq_dataset_validation = boolq_dataset_validation.map(concatenate_question_passage)
print(boolq_dataset_validation[0])

{'question': 'does ethanol take more energy make that produces', 'passage': "Ethanol fuel -- All biomass goes through at least some of these steps: it needs to be grown, collected, dried, fermented, distilled, and burned. All of these steps require resources and an infrastructure. The total amount of energy input into the process compared to the energy released by burning the resulting ethanol fuel is known as the energy balance (or ``energy returned on energy invested''). Figures compiled in a 2007 report by National Geographic Magazine point to modest results for corn ethanol produced in the US: one unit of fossil-fuel energy is required to create 1.3 energy units from the resulting ethanol. The energy balance for sugarcane ethanol produced in Brazil is more favorable, with one unit of fossil-fuel energy required to create 8 from the ethanol. Energy balance estimates are not easily produced, thus numerous such reports have been generated that are contradictory. For instance, a separa

In [91]:
correct = 0
total = len(boolq_dataset_validation)
pipe = pipeline(task="text-generation", model=model_lora_boolq, tokenizer=tokenizer, max_length=200)


for example in boolq_dataset_validation:
    prompt=example['combine']
    result = pipe(prompt, max_length=1, num_return_sequences=1, do_sample=True)
    generated_answer = result[0]['generated_text'][-1]
    if int(generated_answer) == example['label']:
        correct += 1

accuracy = correct / total
print(f"Accuracy: {accuracy}")

/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Accuracy: 0.7128440366972477


## RTE

In [94]:
rte_dataset = load_dataset("super_glue", "rte")

rte_dataset_training=rte_dataset["train"]
print(rte_dataset_training)
rte_dataset_validation=boolq_dataset["validation"]
def concatenate_question_passage(example):
#     answer="True" if example["label"]==1 else "False"
    return {'combine': f"<PREMISE>{example['premise']} </PREMISE> <HYPOTHESIS> {example['hypothesis']} </HYPOTHESIS>{example['label']}"}

rte_dataset_training = rte_dataset_training.map(concatenate_question_passage)
print(rte_dataset_training[0]["combine"])

Dataset({
    features: ['premise', 'hypothesis', 'idx', 'label'],
    num_rows: 2490
})


Map:   0%|          | 0/2490 [00:00<?, ? examples/s]

<PREMISE>No Weapons of Mass Destruction Found in Iraq Yet. </PREMISE> <HYPOTHESIS> Weapons of Mass Destruction Found in Iraq. </HYPOTHESIS>1


In [96]:
training_arguments = TrainingArguments(
    output_dir="./rte_results",
    num_train_epochs=4,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    # change the lr
    learning_rate=2e-5,
    weight_decay=0.001,
    fp16=False,
    #A100 True
    bf16=True,
    max_grad_norm=0.3,
 #   max_steps=max_steps,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)


trainer_rte = SFTTrainer(
    model=model,
    train_dataset=rte_dataset_training,
    peft_config=peft_config,
    dataset_text_field="combine",
#     dataset_text_field=["question", "passage"],
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/2490 [00:00<?, ? examples/s]

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [97]:
trainer_rte.train()
new_model = "llama-2-7b-rte"
trainer_rte.model.save_pretrained(new_model)

/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 2.4682, 'learning_rate': 2e-05, 'epoch': 0.04}


/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 2.4438, 'learning_rate': 2e-05, 'epoch': 0.08}


/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


...

{'loss': 1.4296, 'learning_rate': 2e-05, 'epoch': 3.93}


/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.3087, 'learning_rate': 2e-05, 'epoch': 3.97}


/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'train_runtime': 1347.4381, 'train_samples_per_second': 7.392, 'train_steps_per_second': 1.849, 'train_loss': 1.4564326540424966, 'epoch': 4.0}


In [102]:
# Reload model in FP16 and merge it with LoRA weights
model_lora_rte = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

model_lora_rte = PeftModel.from_pretrained(model_lora_rte, new_model)
model_lora_rte = model_lora_rte.merge_and_unload()
print(id(model_lora_rte))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

46922336938592


In [103]:
rte_dataset_validation=rte_dataset["validation"]
def concatenate_question_passage(example):
#     answer="True" if example["label"]==1 else "False"
    return {'combine': f"<PREMISE>{example['premise']} </PREMISE> <HYPOTHESIS> {example['hypothesis']} </HYPOTHESIS>"}

rte_dataset_validation = rte_dataset_validation.map(concatenate_question_passage)
print(rte_dataset_validation[0])


{'premise': 'Dana Reeve, the widow of the actor Christopher Reeve, has died of lung cancer at age 44, according to the Christopher Reeve Foundation.', 'hypothesis': 'Christopher Reeve had an accident.', 'idx': 0, 'label': 1, 'combine': '<PREMISE>Dana Reeve, the widow of the actor Christopher Reeve, has died of lung cancer at age 44, according to the Christopher Reeve Foundation. </PREMISE> <HYPOTHESIS> Christopher Reeve had an accident. </HYPOTHESIS>'}


In [107]:
correct = 0
total = len(rte_dataset_validation)
pipe = pipeline(task="text-generation", model=model_lora_rte, tokenizer=tokenizer, max_length=200)


for example in rte_dataset_validation:
    prompt=example['combine']
    result = pipe(prompt, max_length=1, num_return_sequences=1, do_sample=True)
#     print(result[0]['generated_text'])
    generated_answer = result[0]['generated_text'][-1]
#     print(generated_answer)
    if int(generated_answer) == example['label']:
        correct += 1

accuracy = correct / total
print(f"Accuracy: {accuracy}")

/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Accuracy: 0.7148014440433214


## WSC

In [41]:
wsc_dataset = load_dataset("super_glue", "wsc")

wsc_dataset_training=wsc_dataset["train"]
print(wsc_dataset_training)
wsc_dataset_validation=wsc_dataset["validation"]
def concatenate_question_passage(example):
#     answer="True" if example["label"]==1 else "False"
    return {'combine': f"<QUESTION>The SPAN1_TEXT <{example['span1_text']}> which index is SPAN1_INDEX <{example['span1_index']}> and SPAN2_TEXT <{example['span2_text']}> which index is SPAN2_INDEX <{example['span2_index']}> are provided for the same target in TEXT <{example['text']}>?</QUESTION>{example['label']}"}

wsc_dataset_training = wsc_dataset_training.map(concatenate_question_passage)
print(wsc_dataset_training[0]["combine"])

Dataset({
    features: ['text', 'span1_index', 'span2_index', 'span1_text', 'span2_text', 'idx', 'label'],
    num_rows: 554
})


Map:   0%|          | 0/554 [00:00<?, ? examples/s]

<QUESTION>The SPAN1_TEXT <Mark> which index is SPAN1_INDEX <0> and SPAN2_TEXT <He> which index is SPAN2_INDEX <13> are provided for the same target in TEXT <Mark told Pete many lies about himself, which Pete included in his book. He should have been more skeptical.>?</QUESTION>0


In [42]:
training_arguments = TrainingArguments(
    output_dir="./wsc_results",
    num_train_epochs=8,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    # checkpoint save step
    save_steps=250,
    logging_steps=25,
    # change the lr
    learning_rate=2e-5,
    weight_decay=0.001,
    fp16=False,
    #A100 True
    bf16=True,
    max_grad_norm=0.3,
 #   max_steps=max_steps,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)


trainer_wsc = SFTTrainer(
    model=model,
    train_dataset=wsc_dataset_training,
    peft_config=peft_config,
    dataset_text_field="combine",
#     dataset_text_field=["question", "passage"],
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/554 [00:00<?, ? examples/s]

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [43]:
trainer_wsc.train()
new_model = "llama-2-7b-wsc"
trainer_wsc.model.save_pretrained(new_model)

/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,3.513800
50,3.070300
75,2.541200
100,1.886000
125,1.575700
150,1.349300
175,1.113700
200,1.119500
225,1.143000
250,0.989400


/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/hice1/jli943/.conda/envs

In [44]:
# Reload model in FP16 and merge it with LoRA weights
new_model = "llama-2-7b-wsc"
model_lora_wsc = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

model_lora_wsc = PeftModel.from_pretrained(model_lora_wsc, new_model)
model_lora_wsc = model_lora_wsc.merge_and_unload()
print(id(model_lora_wsc))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

46925531490528


In [45]:
wsc_dataset_validation=wsc_dataset["validation"]
def concatenate_question_passage(example):
#     answer="True" if example["label"]==1 else "False"
     return {'combine': f"<QUESTION>The SPAN1_TEXT <{example['span1_text']}> which index is SPAN1_INDEX <{example['span1_index']}> and SPAN2_TEXT <{example['span2_text']}> which index is SPAN2_INDEX <{example['span2_index']}> are provided for the same target in TEXT <{example['text']}>?</QUESTION>"}
wsc_dataset_validation = wsc_dataset_validation.map(concatenate_question_passage)
print(wsc_dataset_validation[0])

Map:   0%|          | 0/104 [00:00<?, ? examples/s]

{'text': 'Bernard , who had not told the government official that he was less than 21 when he filed for a homestead claim, did not consider that he had done anything dishonest. Still, anyone who knew that he was 19 years old could take his claim away from him .', 'span1_index': 32, 'span2_index': 47, 'span1_text': 'anyone', 'span2_text': 'him', 'idx': 0, 'label': 0, 'combine': '<QUESTION>The SPAN1_TEXT <anyone> which index is SPAN1_INDEX <32> and SPAN2_TEXT <him> which index is SPAN2_INDEX <47> are provided for the same target in TEXT <Bernard , who had not told the government official that he was less than 21 when he filed for a homestead claim, did not consider that he had done anything dishonest. Still, anyone who knew that he was 19 years old could take his claim away from him .>?</QUESTION>'}


In [46]:
correct = 0
total = len(wsc_dataset_validation)
pipe = pipeline(task="text-generation", model=model_lora_wsc, tokenizer=tokenizer, max_length=200)

notNumber=[]


for example in wsc_dataset_validation:
    prompt=example['combine']
    result = pipe(prompt, max_length=1, num_return_sequences=1, do_sample=True)
#     print(result[0]['generated_text'])
    generated_answer = result[0]['generated_text'][-1]
#     print(generated_answer)
    if generated_answer=="0" or generated_answer=="1":
        if int(generated_answer) == example['label']:
            correct += 1
    else:
        notNumber.append(generated_answer)

accuracy = correct / total
print(f"Accuracy: {accuracy}")
print(notNumber)

/home/hice1/jli943/.conda/envs/multiqlora/lib/python3.8/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Input length of input_ids is 127, but `max_length` is set to 1. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 125, but `max_length` is set to 1. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 95, but `max_length` is set to 1. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 108, but `max_length` is set to 1. This can lead to unexpected behavior. You should c

Input length of input_ids is 75, but `max_length` is set to 1. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 73, but `max_length` is set to 1. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 83, but `max_length` is set to 1. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 130, but `max_length` is set to 1. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 75, but `max_length` is set to 1. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 119, but `max_length` is set to 1. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 90, but `max_length` is set to 1. This can lead to unexpected behavio

Accuracy: 0.5
['2']
